In [169]:
import os

#import libraries for data wrangling
import pandas as pd
import numpy as np

#import libraries for plotting data
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#import libraries for time series analysis
from fbprophet import Prophet

#import libraries to evaluate model performance
import sklearn.metrics as metrics


from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed
from warnings import catch_warnings
from warnings import filterwarnings


In [193]:
train_df, validation_df, test_df = pd.read_csv('data/train_fil_3.csv',index_col=0),pd.read_csv('data/validation_fil_3.csv',index_col=0),pd.read_csv('data/test_fil_3.csv',index_col=0)
for df in [train_df,validation_df,test_df]:
    df.columns = [int(col) for col in df.columns]
    df.index = pd.to_datetime(df.index)

In [179]:
#function to calculate MAPE for all observations where y_true is not 0
def mape(y_true, y_predict):
    '''Returns mean percentage error for all predictions where y_true is not 0. Where y_true is 0, the percentage error is 0 as well '''
    return np.mean([np.absolute(y_true[idx] - y_predict[idx])/y_true[idx] * 100 if y_true[idx] != 0 else 0 for idx,_ in enumerate(y_true) ])

def median_pe(y_true, y_predict):
    '''Returns mean percentage error for all predictions where y_true is not 0. Where y_true is 0, the percentage error is 0 as well '''
    return np.median([np.absolute(y_true[idx] - y_predict[idx])/y_true[idx] * 100 if y_true[idx] != 0 else 0 for idx,_ in enumerate(y_true) ])

def residuals(y_true,y_predict):
    '''Returns list with residuals for all observations where y_true not 0. Where y_true is 0, the residuals are 0 as well '''
    return [y_true[idx] - y_predict[idx] if y_true[idx] != 0 else 0 for idx,_ in enumerate(y_true) ]

def pct_residuals(y_true,y_predict):
    '''Returns list with percentage errors for all observations where y_true not 0. Where y_true is 0, the percentage error is 0 as well'''
    return [(y_true[idx] - y_predict[idx])/y_true[idx] * 100 if y_true[idx] != 0 else 0 for idx,_ in enumerate(y_true) ]

# Plot cumulative density function of residuals
def residual_cdf(data):
    '''Plots cdf of residual input data'''
    # sort the data:
    data_sorted = np.sort(data)

    # calculate the proportional values of samples
    p = 1. * np.arange(len(data)) / (len(data) - 1)

    # plot the sorted data:
    fig = plt.figure(figsize=(20,15))


    ax1 = fig.add_subplot(311)
    ax1.plot(data_sorted, p)
    ax1.set_title('Residuals Cumulative Distribution Function')
    ax1.set_xlabel('Residuals');
    ax1.set_ylabel('Cumulative Distribution');
    ax1.axvline(x=np.percentile(data,5),color='r') 
    ax1.axvline(x=np.percentile(data,95),color='r')

    ax2 = fig.add_subplot(312)
    ax2.plot([idx for idx,_ in enumerate(data)],data,'bo');
    ax2.plot([idx for idx,_ in enumerate(data)],np.zeros(len(data)),'r-');
    ax2.set_title('Residuals over time')
    ax2.set_xlabel('Time in days');
    ax2.set_ylabel('Residual');  
    
    #Here, we could also add Q-Q plot and auto correlation plot for the residual
    
def plot_prediction(y_true,y_predict):
    '''Plots true and predicted values on same y-axis'''
    fig = plt.figure(figsize=(20,15))
    ax1 = fig.add_subplot(311)
    ax1.plot(range(len(y_true)), y_true,'bo')
    ax1.plot(range(len(y_predict)),y_predict,'r-')
    ax1.set_title('Complete prediction')
    
    ax2 = fig.add_subplot(312)
    ax2.plot(range(len(y_true[:60])), y_true[:60],'bo')
    ax2.plot(range(len(y_predict[:60])),y_predict[:60],'r-o')
    ax2.set_title('Prediction first 60 days')
    ax2.set_ylim(0,max(y_true))
    ax3 = fig.add_subplot(313)
    ax3.plot(range(len(y_true[-60:])), y_true[-60:],'bo')
    ax3.plot(range(len(y_predict[-60:])),y_predict[-60:],'ro-')
    ax3.set_title('Prediction last 60 days')
    
def management_summary(y_true,y_predict):
    data = pd.DataFrame.from_dict({'y_true':y_true, 'y_predict':y_predict})
    
    #only regard data where y_true is not 0
    ex_0 = data[data['y_true'] != 0]
    
    #calculate how ofter we under- and over-estimate the revenue
    pct_lower = round(sum(ex_0.y_predict - ex_0.y_true < 0)/len(ex_0.y_true) * 100,1)
    pct_higher = round(100 - pct_lower,1)
    
    #calculate cumulative sums of under- and over estimation
    cumsum_lower = np.cumsum([np.abs(ex_0.y_predict[idx] - ex_0.y_true[idx]) if ex_0.y_predict[idx] < ex_0.y_true[idx] else 0 for idx,y in enumerate(ex_0.y_true) ])
    cumsum_higher = np.cumsum([np.abs(ex_0.y_predict[idx] - ex_0.y_true[idx]) if ex_0.y_predict[idx] > ex_0.y_true[idx] else 0 for idx,y in enumerate(ex_0.y_true)])

    
    fig = plt.figure(figsize=(20,15))
    ax1 = fig.add_subplot(211)
    ax1.plot(range(len(ex_0.y_true)), cumsum_lower,'b-o')
    ax1.plot(range(len(ex_0.y_true)),cumsum_higher,'r-o')
    
    ax1.set_title('Cumulative Sums of Under- and Over-Estimation')
    ax1.set_xlabel('Time')
    ax1.set_ylabel('Cumulated sum of errors')
    ax1.legend(['Under Estimation', 'Over Estimation', 'True Values'])
    
    ax2 = ax1.twinx()
    color = 'black'
    ax2.set_ylabel('Measured Values', color = color)
    ax2.plot(range(len(ex_0.y_true)),ex_0.y_true,'--', color=color, marker=10)
    
    return f'The model underestimates {pct_lower}% of the time'
        

In [171]:
def simple_prophet_forecast(history, cfg):
    '''History must be dataframe with at least the date column "ds" and the y column "y" '''
    	# create config instances
    timesteps,growth,changepoints,n_changepoints,changepoint_range,yearly_seasonality,weekly_seasonality,daily_seasonality,holidays,seasonality_mode,seasonality_prior_scale,holidays_prior_scale,changepoint_prior_scale,mcmc_samples,interval_width,uncertainty_samples = cfg
                                                                    
    model = Prophet(growth=growth,changepoints=changepoints,n_changepoints=n_changepoints,changepoint_range=changepoint_range,yearly_seasonality=yearly_seasonality,weekly_seasonality=weekly_seasonality,daily_seasonality=daily_seasonality,holidays=holidays,seasonality_mode=seasonality_mode,seasonality_prior_scale=seasonality_prior_scale,holidays_prior_scale=holidays_prior_scale,changepoint_prior_scale=changepoint_prior_scale,mcmc_samples=mcmc_samples,interval_width=interval_width,uncertainty_samples=uncertainty_samples)
    model_fit = model.fit(history)
    future = model_fit.make_future_dataframe(timesteps)
    yhat = model_fit.predict(future)
    return yhat[-1:]['yhat'].values[0]

In [172]:
# walk-forward validation for univariate data
def walk_forward_validation(train, test, cfg):
	predictions = list()
	# seed history with training dataset
	history = train.copy()
	# step over each time-step in the test set
	for i in range(len(test)):
		# fit model and make forecast for history
		yhat = simple_prophet_forecast(history, cfg)
		# store forecast in list of predictions
		predictions.append(yhat)
		# add actual observation to history for the next loop
		history.append(test[test.index == test.index[i]])
	# estimate prediction error
	error = mape(test['y'], predictions)
	return error

In [189]:
def walk_forward_prediction(train, test, cfg):
	predictions = list()
	# seed history with training dataset
	history = train.copy()
	# step over each time-step in the test set
	for i in range(len(test)):
		# fit model and make forecast for history
		yhat = simple_prophet_forecast(history, cfg)
		# store forecast in list of predictions
		predictions.append(yhat)
		# add actual observation to history for the next loop
		history.append(test[test.index == test.index[i]])
	# estimate prediction error
	#error = mape(test['y'], predictions)
	return predictions

In [173]:
# score a model, return None on failure
def score_model(train,test, cfg, debug=True):
	result = None
	# convert config to a key
	key = str(cfg)
	# show all warnings and fail on exception if debugging
	if debug:
		result = walk_forward_validation(train,test, cfg)
	else:
		# one failure during model validation suggests an unstable config
		try:
			# never show warnings when grid searching, too noisy
			with catch_warnings():
				filterwarnings("ignore")
				result = walk_forward_validation(train,test, cfg)
		except:
			error = None
	# check for an interesting result
	if result is not None:
		print(' > Model[%s] %.3f' % (key, result))
	return (key, result)

In [177]:
def grid_search(train, test, cfg_list, parallel=False):
	scores = None
	if parallel:
		# execute configs in parallel
		executor = Parallel(n_jobs=cpu_count(), backend='multiprocessing')
		tasks = (delayed(score_model)(train,test, cfg) for cfg in cfg_list)
		scores = executor(tasks)
	else:
		scores = [score_model(train,test, cfg) for cfg in cfg_list]
	# remove empty results
	scores = [r for r in scores if r[1] != None]
	# sort configs by error, asc
	scores.sort(key=lambda tup: tup[1])
	return scores

In [175]:
# create a set of sarima configs to try
def prophet_configs():
	models = list()
	# define config lists
	timesteps = [1]
	growth=['linear']
	changepoints=[None]
	n_changepoints=[25]
	changepoint_range=[0.8]
	yearly_seasonality=[True]
	weekly_seasonality=[True]
	daily_seasonality=['auto']
	holidays=[None]
	seasonality_mode=['additive','multiplicative']
	seasonality_prior_scale=[10.0,20.0]
	holidays_prior_scale=[10.0]
	changepoint_prior_scale=[0.05]
	mcmc_samples=[0,1]
	interval_width=[0.8]
	uncertainty_samples=[100]

	# create config instances
	for t in timesteps:
		for g in growth:
			for c in changepoints:
				for nc in n_changepoints:
					for rc in changepoint_range:
						for ys in yearly_seasonality:
							for ws in weekly_seasonality:
								for ds in daily_seasonality:
									for h in holidays:
										for sm in seasonality_mode:
											for sps in seasonality_prior_scale:
												for hps in holidays_prior_scale:
													for cps in changepoint_prior_scale:
														for mcmc in mcmc_samples:
															for iw in interval_width:
																for us in uncertainty_samples:
																	cfg = [t,g,c,nc,rc,ys,ws,ds,h,sm,sps,hps,cps,mcmc,iw,us]
																	models.append(cfg)
	return models

In [178]:
if __name__ == '__main__':
	# define dataset
	train = pd.DataFrame.from_dict({'ds':train_df.index,'y':train_df[6]})
	test = pd.DataFrame.from_dict({'ds':validation_df.index,'y':validation_df[6]})
	# model configs
	cfg_list = prophet_configs()
	# grid search
	scores = grid_search(train, test, cfg_list)
	print('done')
	# list top 3 configs
	for cfg, error in scores[:3]:
	    print(cfg, error)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/Users/leo/anaconda3/envs/nf/lib/python3.6/site-packages/pystan/misc.py:399: FutureWarning:

Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling dai

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

...

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

 > Model[[1, 'linear', None, 25, 0.8, True, True, 'auto', None, 'multiplicative', 20.0, 10.0, 0.05, 1, 0.8, 100]] 117.460
done
[1, 'linear', None, 25, 0.8, True, True, 'auto', None, 'multiplicative', 10.0, 10.0, 0.05, 0, 0.8, 100] 33.158621864720494
[1, 'linear', None, 25, 0.8, True, True, 'auto', None, 'multiplicative', 20.0, 10.0, 0.05, 0, 0.8, 100] 33.2515169520717
[1, 'linear', None, 25, 0.8, True, True, 'auto', None, 'additive', 10.0, 10.0, 0.05, 0, 0.8, 100] 34.33182574315419


In [191]:
pred = walk_forward_prediction(train,test,[1, 'linear', None, 25, 0.8, True, True, 'auto', None, 'multiplicative', 10.0, 10.0, 0.05, 0, 0.8, 1])

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

In [192]:
pred

[279.65358836100637,
 279.65358836100637,
 279.65358836100637,
 279.65358836100637,
 279.65358836100637,
 279.65358836100637,
 279.65358836100637,
 279.65358836100637,
 279.65358836100637,
 279.65358836100637,
 279.65358836100637,
 279.65358836100637,
 279.65358836100637,
 279.65358836100637,
 279.65358836100637,
 279.65358836100637,
 279.65358836100637,
 279.65358836100637,
 279.65358836100637,
 279.65358836100637,
 279.65358836100637,
 279.65358836100637,
 279.65358836100637,
 279.65358836100637,
 279.65358836100637,
 279.65358836100637,
 279.65358836100637,
 279.65358836100637,
 279.65358836100637,
 279.65358836100637,
 279.65358836100637,
 279.65358836100637,
 279.65358836100637,
 279.65358836100637,
 279.65358836100637,
 279.65358836100637,
 279.65358836100637,
 279.65358836100637,
 279.65358836100637,
 279.65358836100637,
 279.65358836100637,
 279.65358836100637,
 279.65358836100637,
 279.65358836100637,
 279.65358836100637,
 279.65358836100637,
 279.65358836100637,
 279.65358836